# Geospatial Matching Optimization

This project is designed to showcase how to improve the speed of a polygon matching procedure. There are two main areas that could benefit from taking a "geospatial-native" approach:

1. Data encoding, fetching, and storage
2. Matching

We will generate synthetic polygon data and use both a non-geospatial-native and geospatial-native approach, comparing execution time for both to see how they differ.

Note: Make sure that you have PostgreSQL running.

## Imports

In [597]:
import geopandas as gpd
import pandas as pd
import shapely
import h3
import time
import helpers as h
import matplotlib.pyplot as plt
from shapely.wkt import dumps, loads
from pathlib import Path
import multiprocessing as mp

# ensure helpers is loaded correctly
import importlib
importlib.reload(h)

<module 'helpers' from '/Users/sra/files/projects/matching_optimization/helpers.py'>

## Generate Test Data

In [598]:
# Create two blobs of data
prev_month_blobs = h.generate_random_polygons(n=10000)
curr_month_blobs = h.generate_random_polygons(n=10000)

Function `generate_random_polygons` executed in 7.9995 sec, CPU: 12.20%, Memory: 100.91MB
Function `generate_random_polygons` executed in 8.0313 sec, CPU: 31.50%, Memory: 75.55MB


In [599]:
prev_month_blobs.head()

,geometry,id,geohash
885,"POLYGON ((-81.74629 27.21993, -81.74629 27.220...",f1ea62f8-8a4e-4287-a466-08ec0c317ad3,dhvfgg61
5442,"POLYGON ((-81.24467 27.52294, -81.24467 27.523...",9385f950-5f40-438a-8a04-924184052eb6,dhyhx4rt
7700,"POLYGON ((-81.33568 27.52908, -81.33568 27.529...",b3b43333-08ba-40fb-b71d-fdf41e889b86,dhyht5t4
3916,"POLYGON ((-80.97548 27.64739, -80.97548 27.647...",6db32f10-668e-49f5-8dda-a5a813395337,dhymm35d
2040,"POLYGON ((-81.07487 27.73192, -81.07487 27.732...",a2a6d1e6-ebc9-4e68-b628-0ee51f194802,dhymg07p


In [600]:
curr_month_blobs.head()

,geometry,id,geohash
8489,"POLYGON ((-98.43519 26.76248, -98.43519 26.762...",45e67537-93b7-4ca3-aa28-58e581e005d0,9uf00pcx
5469,"POLYGON ((-98.59041 26.96246, -98.59041 26.962...",951a24eb-1c74-4c49-84f7-99776992f246,9uccks2u
5637,"POLYGON ((-98.43162 27.01086, -98.43162 27.011...",f738c3b4-b97f-4a3d-a468-bbc87057472c,9uf18jk8
2809,"POLYGON ((-98.52477 27.15353, -98.52477 27.154...",d8f96695-b930-4954-90c0-4da9d7d6b47f,9ucfqp0x
8606,"POLYGON ((-98.52997 27.20496, -98.52997 27.205...",73867cdf-2752-4629-b673-f574a3e514ef,9ucfvbsc


In [601]:
# make data folder
Path('data/').mkdir(exist_ok=True)

# save as geoparquet
prev_month_blobs.to_parquet('data/prev_month_blobs.parquet')
curr_month_blobs.to_parquet('data/curr_month_blobs.parquet')

In [602]:
# range_ = 10000

# fig, ax = plt.subplots(figsize=(5, 5))
# prev_month_blobs.iloc[0:range_].plot(ax=ax)
# ax.set_title(f'Viewing the first {range_} blobs')
# plt.tight_layout()

# plt.show()

In [603]:
prev_month_blobs.head()

,geometry,id,geohash
885,"POLYGON ((-81.74629 27.21993, -81.74629 27.220...",f1ea62f8-8a4e-4287-a466-08ec0c317ad3,dhvfgg61
5442,"POLYGON ((-81.24467 27.52294, -81.24467 27.523...",9385f950-5f40-438a-8a04-924184052eb6,dhyhx4rt
7700,"POLYGON ((-81.33568 27.52908, -81.33568 27.529...",b3b43333-08ba-40fb-b71d-fdf41e889b86,dhyht5t4
3916,"POLYGON ((-80.97548 27.64739, -80.97548 27.647...",6db32f10-668e-49f5-8dda-a5a813395337,dhymm35d
2040,"POLYGON ((-81.07487 27.73192, -81.07487 27.732...",a2a6d1e6-ebc9-4e68-b628-0ee51f194802,dhymg07p


## Non-optimized process

### 1. Data encoding, storage, and fetching

We will use PostgreSQL and string storage for geographic information.

#### Make copy of polygon layer and convert to non-geospatial-native string (WKT) datatype

In [604]:
# copy the gdfs
prev_month_blobs_wkt = prev_month_blobs.copy()
curr_month_blobs_wkt = curr_month_blobs.copy()

In [605]:
# Convert WKT versions to strings
dfs_to_convert = [prev_month_blobs_wkt, curr_month_blobs_wkt]
prev_month_blobs_wkt, curr_month_blobs_wkt = [h.convert_col_to_string(df) for df in dfs_to_convert]

# Check result
print(prev_month_blobs_wkt.head())

Function `convert_col_to_string` executed in 0.0513 sec, CPU: 11.00%, Memory: 0.17MB
Function `convert_col_to_string` executed in 0.0524 sec, CPU: 59.50%, Memory: 55.05MB
                                               geometry  \
885   POLYGON ((-81.746292 27.21993, -81.746292 27.2...   
5442  POLYGON ((-81.24467 27.522945, -81.24467 27.52...   
7700  POLYGON ((-81.335681 27.52908, -81.335681 27.5...   
3916  POLYGON ((-80.975485 27.647386, -80.975485 27....   
2040  POLYGON ((-81.074867 27.731919, -81.074867 27....   

                                        id   geohash  
885   f1ea62f8-8a4e-4287-a466-08ec0c317ad3  dhvfgg61  
5442  9385f950-5f40-438a-8a04-924184052eb6  dhyhx4rt  
7700  b3b43333-08ba-40fb-b71d-fdf41e889b86  dhyht5t4  
3916  6db32f10-668e-49f5-8dda-a5a813395337  dhymm35d  
2040  a2a6d1e6-ebc9-4e68-b628-0ee51f194802  dhymg07p  


/Users/sra/files/projects/matching_optimization/helpers.py:305: UserWarning: Geometry column does not contain geometry.
  return df
/Users/sra/files/projects/matching_optimization/helpers.py:305: UserWarning: Geometry column does not contain geometry.
  return df


### Save in PostgreSQL database

This is a simple version of what a "non-geospatial-native" data ingestion pipeline may look like, with the primary example being that the polygons are stored as strings, not in a spatial-friendly datatype.

In [606]:
# Convert to tuples
prev_month_blobs_wkt = h.df_itertuple(prev_month_blobs_wkt)
curr_month_blobs_wkt = h.df_itertuple(curr_month_blobs_wkt)

Function `df_itertuple` executed in 0.0031 sec, CPU: 16.70%, Memory: 0.05MB
Function `df_itertuple` executed in 0.0026 sec, CPU: 0.00%, Memory: 0.00MB


Create PostgreSQL database if it doesn't exist already. We will be using the default settings. If they need to be adjusted, refer to [`helpers.py`](helpers.py).

In [607]:
h.create_pg_db()

Database blob_matching already exists.
Function `create_pg_db` executed in 0.0783 sec, CPU: 0.00%, Memory: 0.00MB


Create and insert into tables:

In [608]:
prev_month_blobs.head(1)

,geometry,id,geohash
885,"POLYGON ((-81.74629 27.21993, -81.74629 27.220...",f1ea62f8-8a4e-4287-a466-08ec0c317ad3,dhvfgg61


In [609]:
prev_month_blobs_wkt[0]

('POLYGON ((-81.746292 27.21993, -81.746292 27.220622, -81.746108 27.220622, -81.746108 27.21993, -81.746292 27.21993))',
 'f1ea62f8-8a4e-4287-a466-08ec0c317ad3',
 'dhvfgg61')

In [610]:
h.create_pg_table(table_name='prev_blobs_wkt', data=prev_month_blobs_wkt, truncate=True)
h.create_pg_table(table_name='curr_blobs_wkt', data=curr_month_blobs_wkt, truncate=True)

Table prev_blobs_wkt truncated.
Inserted 10000 records into prev_blobs_wkt.
Function `create_pg_table` executed in 0.4114 sec, CPU: 0.00%, Memory: 0.00MB
Table curr_blobs_wkt truncated.
Inserted 10000 records into curr_blobs_wkt.
Function `create_pg_table` executed in 0.4642 sec, CPU: 39.20%, Memory: 45.73MB


Retrieve data as GeoDataFrames to confirm that it worked:

In [611]:
df_prev = h.retrieve_pg_table(table_name='prev_blobs_wkt')
df_curr = h.retrieve_pg_table(table_name='curr_blobs_wkt')

Retrieved 10000 records from prev_blobs_wkt.
Function `retrieve_pg_table` executed in 0.0504 sec, CPU: 9.50%, Memory: 0.95MB
Retrieved 10000 records from curr_blobs_wkt.
Function `retrieve_pg_table` executed in 0.0446 sec, CPU: 0.00%, Memory: 0.00MB


Compare the tables before and after for a sanity check:

In [612]:
df_prev.head()

,geometry,id,geohash
0,"POLYGON ((-81.746292 27.21993, -81.746292 27.2...",f1ea62f8-8a4e-4287-a466-08ec0c317ad3,dhvfgg61
1,"POLYGON ((-81.24467 27.522945, -81.24467 27.52...",9385f950-5f40-438a-8a04-924184052eb6,dhyhx4rt
2,"POLYGON ((-81.335681 27.52908, -81.335681 27.5...",b3b43333-08ba-40fb-b71d-fdf41e889b86,dhyht5t4
3,"POLYGON ((-80.975485 27.647386, -80.975485 27....",6db32f10-668e-49f5-8dda-a5a813395337,dhymm35d
4,"POLYGON ((-81.074867 27.731919, -81.074867 27....",a2a6d1e6-ebc9-4e68-b628-0ee51f194802,dhymg07p


In [613]:
prev_month_blobs.head()

,geometry,id,geohash
885,"POLYGON ((-81.74629 27.21993, -81.74629 27.220...",f1ea62f8-8a4e-4287-a466-08ec0c317ad3,dhvfgg61
5442,"POLYGON ((-81.24467 27.52294, -81.24467 27.523...",9385f950-5f40-438a-8a04-924184052eb6,dhyhx4rt
7700,"POLYGON ((-81.33568 27.52908, -81.33568 27.529...",b3b43333-08ba-40fb-b71d-fdf41e889b86,dhyht5t4
3916,"POLYGON ((-80.97548 27.64739, -80.97548 27.647...",6db32f10-668e-49f5-8dda-a5a813395337,dhymm35d
2040,"POLYGON ((-81.07487 27.73192, -81.07487 27.732...",a2a6d1e6-ebc9-4e68-b628-0ee51f194802,dhymg07p


In [614]:
def round_geometry(geom, precision=6):
    """Round all coordinates of a geometry to a given precision."""
    return shapely.wkt.loads(shapely.wkt.dumps(geom, rounding_precision=precision))

# Convert both to sets of rounded WKT strings
set_prev_month_blobs = set(prev_month_blobs['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))
set_df_prev = set(df_prev['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))

# Find common, missing, and extra geometries
common_geometries = set_prev_month_blobs & set_df_prev
missing_from_retrieved = set_prev_month_blobs - set_df_prev
extra_in_retrieved = set_df_prev - set_prev_month_blobs

# Print summary
print(f"Number of matching geometries: {len(common_geometries)}")
print(f"Missing geometries in retrieved table: {len(missing_from_retrieved)}")
print(f"Extra geometries in retrieved table: {len(extra_in_retrieved)}")

# Show an example missing/extra geometry for debugging
if missing_from_retrieved:
    print("Example missing record:", next(iter(missing_from_retrieved)))

if extra_in_retrieved:
    print("Example extra record:", next(iter(extra_in_retrieved)))

Number of matching geometries: 10000
Missing geometries in retrieved table: 0
Extra geometries in retrieved table: 0


It worked!

### 2. Matching

We will match the polygons using GeoPandas.

In [615]:
postgresql_details = h.pg_details()
h.run_parallel_matching(table_prev='prev_blobs_wkt', 
                        table_curr='curr_blobs_wkt', 
                        output_table='matched_results', 
                        postgresql_details=postgresql_details, 
                        db_name='blob_matching', 
                        num_workers=4, 
                        batch_size=100)

Table matched_results created successfully.
Retrieved 10000 records from prev_blobs_wkt.


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 936, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1042, in get_code
KeyboardInterrupt
Traceback (most recent call last):
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "<string>", line 1, in <module>
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/python3.11/multiprocessing/spawn.py", l

KeyboardInterrupt: 

geohash polygon into a larger region
match blobs using multiprocessing (mp.Process) and geopandas
find blobs from previous month that do not have a match

# 1 imports and setup

```python
import os, sys, traceback
import argparse
import cv2
import time
import numpy as np
import pandas as pd
import geopandas as gpd
from skimage import io, color, measure
from tqdm import tqdm
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.validation import make_valid
import PIL
from PIL import Image, ImageDraw, ImageEnhance
import uuid
import multiprocessing as mp
from datetime import datetime
from sqlalchemy import or_
from sqlmodel import SQLModel, Session, create_engine, select
from pathlib import Path
```

## Key takeaways
- Uses Pandas and GeoPandas for data processing.
- Uses Shapely for geospatial geometry operations.
- Uses multiprocessing (mp) to parallelize the blob-matching process.
- Uses SQLAlchemy and SQLModel for database operations.

# 2 High-level script overview
The script is designed to match blobs (spatial objects) between two months and classify them into different business categories.

General Workflow
1. Fetch blobs from the previous and current months.
1. Convert the region of interest (state, city, county, or geohash) into a list of geohashes.
1. Parallelized Matching Process:
1. Process multiple geohashes at once using multiprocessing.
1. Find corresponding blobs from previous months for each geohash.
1. Classify blobs based on construction stage progression.
1. Identify blobs that are missing in the current month and "impute" them.
1. Save results into the database.

# 3 Blob classification logic

# 4 Blob matching

- Matches blobs between months using polygon intersections.
- Uses Shapely to validate geometries and check for overlaps.
- Returns matched blob IDs.

## Multiprocessing Optimization
The script parallelizes blob matching by:
- Uses multiprocessing (mp.Process) to divide the dataset into smaller batches.
- Each batch of blobs is processed in parallel.
- Reduces runtime compared to a single-threaded approach.

# 5 Handling unmatched blobs
- Finds blobs from the previous month that do not have a match in the current month.
- These blobs are "imputed", meaning they are carried over into the new month.

# 6 Main class to orchestrate process
- Loads previous and current month blobs.
- Converts input regions (state, city, county) into geohashes.

# Summary
- Parallel processing is used to match blobs.
- Blobs are classified based on construction stages.
- Unmatched blobs are imputed for continuity.
- The script writes results to a database.